In [ ]:
!git clone https://github.com/Taka0007/Re_Impre_arXiv_1706.10059.git
%cd Re_Impre_arXiv_1706.10059

Cloning into 'Re_Impre_arXiv_1706.10059'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 169 (delta 82), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (169/169), 45.56 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (82/82), done.
/content/Re_Impre_arXiv_1706.10059


In [ ]:
!pip install -r requirements.txt -q

In [ ]:
import json
import torch
from data.downloader import download
from data.preprocess import price_tensor
from torch.utils.data import TensorDataset
from pgportfolio.utils.tensor import to_tensor
from pgportfolio.model.cnn import EIIECNN
from pgportfolio.model.rnn import EIIRNN
from pgportfolio.model.lstm import EIILSTM
from pgportfolio.model.pvm import PortfolioVectorMemory
from pgportfolio.core.trainer import Trainer
from pgportfolio.learner.reward import immediate_reward
from pgportfolio.learner.osbl import get_random_loader
from pgportfolio.core.backtester import backtest

In [ ]:
def load_config(path):
    return json.load(open(path))

def build_model(cfg, assets):
    if cfg["model"] == "cnn":
        return EIIECNN(assets, cfg["window"])
    if cfg["model"] == "rnn":
        return EIIRNN(assets, 64)
    if cfg["model"] == "lstm":
        return EIILSTM(assets, 64)

In [ ]:
config = load_config("configs/default.json")

df = download(["BTC-USD","ETH-USD","XRP-USD"], start="2019-01-01", end="2025-01-01")
X, Y = price_tensor(df, config["window"])

xs = to_tensor(X)
ys = to_tensor(Y)

/content/Re_Impre_arXiv_1706.10059/data/downloader.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbols, start=start, end=end)["Close"]
[*********************100%***********************]  3 of 3 completed


In [ ]:
dataset = TensorDataset(xs, ys)
loader = get_random_loader(dataset, config["batch_size"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_model(config, xs.shape[2]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

trainer = Trainer(model, optimizer, immediate_reward, config)

In [ ]:
for epoch in range(config["train_epochs"]):
    avg = trainer.train(loader, device)
    print(f"Epoch {epoch+1}, avg reward={avg:.5f}")

100%|██████████| 34/34 [00:00<00:00, 78.48it/s]


Epoch 1, avg reward=0.00154


100%|██████████| 34/34 [00:00<00:00, 226.62it/s]


Epoch 2, avg reward=0.00158


100%|██████████| 34/34 [00:00<00:00, 227.83it/s]


...

Epoch 98, avg reward=0.00450


100%|██████████| 34/34 [00:00<00:00, 153.58it/s]


Epoch 99, avg reward=0.00453


100%|██████████| 34/34 [00:00<00:00, 153.02it/s]

Epoch 100, avg reward=0.00459


In [ ]:
final_wealth = backtest(model, xs, ys, device, config["commission"])
print("Final wealth:", final_wealth)

Final wealth: 9838.32236719626
